# 1.Original model prediction

#step1

In [1]:
import os
import pandas as pd
import numpy as np
import argparse

#%%
def get_the_index_of_the_seq(source, target):
    tar_len = len(target)
    print('tar_len:',tar_len)
    if (source is None or target is None):
        return -1
    for i in range(len(source) - len(target) + 1):
        if target == source[i:i+tar_len]:
            return i, i+tar_len
        else:
            pass

def identify_begin_and_end_of_each_CDR3(vdj_file_path, 
                                        ID = None,
                                        cdr3b = None, 
                                        cdr3a = None, 
                                        seqb = None, 
                                        seqa = None, 
                                        output_table = None):
    DF = pd.read_csv(vdj_file_path)
    cdr3b_begin_list = list()
    cdr3b_end_list = list()
    cdr3a_begin_list = list()
    cdr3a_end_list = list()
    index_list = list()
    index_DF = pd.DataFrame()
    
    for index, row in DF.iterrows():
        # print('index',index)
        # print('row',row)
        try:   
            cdr3b_begin, cdr3b_end = get_the_index_of_the_seq(source = row[seqb], target = row[cdr3b]) 
            print('cdr3b_begin:',cdr3b_begin)
            cdr3a_begin, cdr3a_end = get_the_index_of_the_seq(source = row[seqa], target = row[cdr3a]) 
            cdr3b_begin_list.append(cdr3b_begin)
            cdr3b_end_list.append(cdr3b_end)
            cdr3a_begin_list.append(cdr3a_begin)
            cdr3a_end_list.append(cdr3a_end)
            index_list.append(row[ID])
        except:
            print('-'*10+'Error'+'-'*10)
            print(row[ID])
            print(row[seqb])
            print(row[cdr3b])
            print(row[seqa])
            print(row[cdr3a])    
    
    index_DF[ID] = index_list                
    index_DF[cdr3a+'_begin'] = cdr3a_begin_list
    index_DF[cdr3a+'_end'] = cdr3a_end_list
    index_DF[cdr3b+'_begin'] = cdr3b_begin_list
    index_DF[cdr3b+'_end'] = cdr3b_end_list    
    Output_DF = DF.merge(index_DF, on=ID, how='inner')
    
    Output_DF.to_csv(output_table, index=False)

## make the DF light/alpha and heavy/beta to fasta file
def get_fasta_file_from_dataframe(DF,
                                  seqb = None,
                                  seqa = None,
                                  seqa_id = None,
                                  seqb_id = None, 
                                  output_fasta_path = None):
    
    seqb_list = DF[seqb].to_list()
    seqa_list = DF[seqa].to_list()  

    for i in range(len(DF)):
        # write the seqa fasta file
        with open(output_fasta_path+seqa_id+'.fasta', 'a') as f:
            f.write('>'+DF.loc[i , seqa_id]  +'\n'+seqa_list[i]+'\n')
        # write the seqb fasta file
        with open(output_fasta_path+seqb_id+'.fasta', 'a') as f:
            f.write('>'+DF.loc[i , seqb_id]  +'\n'+seqb_list[i]+'\n')


## make the light/alpha and heavy/beta to fasta file for input of the AF2
def get_fasta_file(df_path,
                   ID = None,
                   cdr3b = None, 
                   cdr3a = None, 
                   seqb = None, 
                   seqa = None,
                   output_fasta_path = None):
    
    DF = pd.read_csv(df_path)
    seqb_list = DF[seqb].to_list()
    seqa_list = DF[seqa].to_list()
   
    # make the output fasta file path
    os.makedirs(os.path.join(output_fasta_path),exist_ok = True)
    os.makedirs(os.path.join(output_fasta_path),exist_ok = True)

    for i in range(len(DF)):
        output_CDR3b_ID_note = f'{DF.loc[i , cdr3b]}_{DF.loc[i , ID]}'
        output_CDR3a_ID_note = f'{DF.loc[i , cdr3a]}_{DF.loc[i , ID]}'
        # write the seqa fasta file
        with open(os.path.join(output_fasta_path ,  output_CDR3a_ID_note+".fasta"), 'a') as f:
            f.write('>'+output_CDR3a_ID_note  +'\n'+seqa_list[i]+'\n')
        # write the seqb fasta file
        with open(os.path.join(output_fasta_path ,  output_CDR3b_ID_note+".fasta"), 'a') as f:
            f.write('>'+output_CDR3b_ID_note  +'\n'+seqb_list[i]+'\n')


In [ ]:

testfile_path=f"../../test.csv"

df=pd.read_csv(testfile_path)
df['ID']=list(range(len(df)))
AIR_file_path=f"./out_folder_step1/test_ID.csv"
os.makedirs('./out_folder_step1/', exist_ok=True)
df.to_csv(AIR_file_path, index=False)

output_table = './out_folder_step1/CDR3Region.csv'
output_fasta_folder = './out_folder_step1/fasta'        
os.makedirs(output_fasta_folder, exist_ok=True)
ID = 'ID'
Chain1 = 'LongB'
Chain2 = 'LongA'
Chain1_cdr3 = 'CDR3B'
Chain2_cdr3 = 'CDR3A'
identify_begin_and_end_of_each_CDR3(AIR_file_path, 
                                    ID = ID,
                                    cdr3b = Chain1_cdr3, 
                                    cdr3a = Chain2_cdr3, 
                                    seqb = Chain1, 
                                    seqa = Chain2, 
                                    output_table = output_table
                                    )
get_fasta_file(df_path = output_table,
                ID = ID,
                cdr3b = Chain1_cdr3, 
                cdr3a = Chain2_cdr3, 
                seqa = Chain2,
                seqb = Chain1,
                output_fasta_path = output_fasta_folder
                )

#step2

#Run the step2.sh script in this directory to generate sequence structure information using AlphaFold, which will be used as the input for step3.

#step3

In [ ]:
    
import os
import pandas as pd
import numpy as np
import pickle
import argparse
import time
#%%
def get_AF2_feature(feature_file):
    time.sleep(0.1)
    with open(feature_file,'rb') as f:
        feature = pickle.load(f)
    return feature['representations']['structure_module']


def parse_AF2_data(AIR_file_path, 
                    AF2_feature_folder, 
                    output_folder, 
                    max_len = 40,
                    feature_dimentation = 384,
                    ID = 'ID', 
                    CDR3b = 'Heavy_cdr3', 
                    CDR3a = 'Light_cdr3', 
                    b_vgene = 'Heavy_v_gene',
                    b_jgene = 'Heavy_j_gene',
                    a_vgene = 'Light_v_gene',
                    a_jgene = 'Light_j_gene',
                    cdr3b_begin = 'Heavy_cdr3_begin', 
                    cdr3b_end = 'Heavy_cdr3_end',
                    cdr3a_begin = 'Light_cdr3_begin',
                    cdr3a_end = 'Light_cdr3_end',
                    af_pkl_file = 'result_model_1_ptm_pred_0.pkl',
                    task_name = '*'
                    ):
    
    os.makedirs(output_folder, exist_ok=True)
    
    AIR_df = pd.read_csv(AIR_file_path)
    
    for index, row in AIR_df.iterrows():
        print('-'*30)
        print(f'{index}_{row[CDR3b]}')
        print('-'*30)
                    
        current_CDR3b_ID_note = f'{row[CDR3b]}_{row[ID]}'
        current_CDR3a_ID_note = f'{row[CDR3a]}_{row[ID]}'
        
        current_cdr3b_begin = row[cdr3b_begin]
        current_cdr3b_end = row[cdr3b_end]
        current_cdr3a_begin = row[cdr3a_begin]
        current_cdr3a_end = row[cdr3a_end]

        # B chain
        CDR3b_Feature = np.zeros((max_len, feature_dimentation))
        feature_file = os.path.join(AF2_feature_folder, current_CDR3b_ID_note, af_pkl_file)
        # print('feature_file:',feature_file)
        CDR3b_Feature[0:(current_cdr3b_end-current_cdr3b_begin)] = get_AF2_feature(feature_file)[current_cdr3b_begin:current_cdr3b_end]
        
        # A chain
        CDR3a_Feature = np.zeros((max_len, feature_dimentation))
        feature_file = os.path.join(AF2_feature_folder, current_CDR3a_ID_note, af_pkl_file)
        CDR3a_Feature[0:(current_cdr3a_end-current_cdr3a_begin)] = get_AF2_feature(feature_file)[current_cdr3a_begin:current_cdr3a_end]            
        
        output_CDR3b_ID_note = f'{row[CDR3b]}_{row[b_vgene].replace("/", "-")}_{row[b_jgene].replace("/", "-")}_{row[ID]}_{task_name}.npy'
        output_CDR3a_ID_note = f'{row[CDR3a]}_{row[a_vgene].replace("/", "-")}_{row[a_jgene].replace("/", "-")}_{row[ID]}_{task_name}.npy'   
        np.save(os.path.join(output_folder, output_CDR3b_ID_note), CDR3b_Feature)
        np.save(os.path.join(output_folder, output_CDR3a_ID_note), CDR3a_Feature)     
        
        print(output_CDR3b_ID_note)
        print(CDR3b_Feature)
        print(output_CDR3a_ID_note)
        print(CDR3a_Feature)



In [ ]:
if __name__ == '__main__':
    AIR_file_path ='./out_folder_step1/CDR3Region.csv'
    AF2_feature_folder = f'./alphafold/output_af2_structure/'  
    output_folder = f'./out_folder_step3/'
    ID = 'ID'
    Chain1_cdr3 = 'CDR3B'
    Chain2_cdr3 = 'CDR3A'
    Chain1_v_gene = 'TRBV'
    Chain1_j_gene = 'TRBJ'
    Chain2_v_gene = 'TRAV'
    Chain2_j_gene = 'TRAJ'
    task_name = 'unseen'
    
    parse_AF2_data(AIR_file_path, 
                    AF2_feature_folder, 
                    output_folder, 
                    max_len = 40,
                    feature_dimentation = 384,
                    ID = ID, 
                    CDR3b = Chain1_cdr3, 
                    CDR3a = Chain2_cdr3, 
                    b_vgene = Chain1_v_gene,
                    b_jgene = Chain1_j_gene,
                    a_vgene = Chain2_v_gene,
                    a_jgene = Chain2_j_gene,
                    cdr3b_begin = f'{Chain1_cdr3}_begin', 
                    cdr3b_end = f'{Chain1_cdr3}_end',
                    cdr3a_begin = f'{Chain2_cdr3}_begin',
                    cdr3a_end = f'{Chain2_cdr3}_end',
                    task_name = task_name
                    )

#Finally, run the DeepAIR Jupyter notebook in the `maincode` folder to obtain the final results.